In [1]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True
from transformers import BertModel, BertTokenizer

# from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit


[2024-02-06 22:54:05,010] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
model_path = "/home/toibazd/Data/BERT/BERT_context_pretrained_InterPro_final"

model = BertModel.from_pretrained(model_path)
model.pooler = None
model.get_output_embeddings()
model.eval()

Some weights of BertModel were not initialized from the model checkpoint at /home/toibazd/Data/BERT/BERT_context_pretrained_InterPro_final and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(544998, 256, padding_idx=0)
    (position_embeddings): Embedding(512, 256)
    (token_type_embeddings): Embedding(2, 256)
    (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=256, out_features=256, bias=True)
            (key): Linear(in_features=256, out_features=256, bias=True)
            (value): Linear(in_features=256, out_features=256, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=256, out_features=256, bias=True)
            (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [11]:
# tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
# Load pre-trained BERT tokenizer from local file and add special tokens
tokenizer = BertTokenizer.from_pretrained(
    "WordLevel_tokenizer_trained_InterPro.json",
    bos_token="[CLS]",
    eos_token="[SEP]",
    unk_token="[UNK]",
    pad_token="[PAD]",
    mask_token="[MASK]"
)
tokenizer.pre_tokenizer = WhitespaceSplit()
# tokenizer_path = "WordLevel_tokenizer_trained_InterPro.json"
# tokenizer = tokenizer.from_file(tokenizer_path)
# tokenizer.enable_truncation(512)
tokenizer.vocab_size


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


545063

In [12]:
tokenizer.get_vocab()

{'{': 0,
 '  "version": "1.0",': 1,
 '  "truncation": null,': 2,
 '  "padding": null,': 3,
 '  "added_tokens": [': 4,
 '    {': 41,
 '      "id": 0,': 6,
 '      "content": "[UNK]",': 7,
 '      "single_word": false,': 44,
 '      "lstrip": false,': 45,
 '      "rstrip": false,': 46,
 '      "normalized": false,': 47,
 '      "special": true': 48,
 '    },': 545132,
 '      "id": 1,': 15,
 '      "content": "[CLS]",': 16,
 '      "id": 2,': 24,
 '      "content": "[SEP]",': 25,
 '      "id": 3,': 33,
 '      "content": "[PAD]",': 34,
 '      "id": 4,': 42,
 '      "content": "[MASK]",': 43,
 '    }': 128,
 '  ],': 50,
 '  "normalizer": null,': 51,
 '  "pre_tokenizer": {': 52,
 '    "type": "WhitespaceSplit"': 53,
 '  },': 129,
 '  "post_processor": {': 55,
 '    "type": "TemplateProcessing",': 56,
 '    "single": [': 57,
 '      {': 102,
 '        "SpecialToken": {': 103,
 '          "id": "[CLS]",': 80,
 '          "type_id": 0': 93,
 '        }': 106,
 '      },': 118,
 '        "Seq

In [5]:
import pandas as pd


df = pd.read_csv('KOs_with_labels.csv')
label_to_int_mapping = {label: i for i, label in enumerate(df['label'].unique())}
ko_label_dict = {}
for index, row in df.iterrows():
    ko, label = row['KOs'], row['label']
    if ko not in ko_label_dict:
        ko_label_dict[ko] = label
df = df.drop(columns=["Unnamed: 0"])
print(df.head(10))
print("Length of dataframe = ", len(df))

        KOs                                        label
0  K02868.1                                     Ribosome
1  K02987.1                                     Ribosome
2  K00571.1                   Prokaryotic defense system
3  K02978.1                                     Ribosome
4  K02929.2                                     Ribosome
5  K22011.1         Porphyrin and chlorophyll metabolism
6  K02896.3                                     Ribosome
7  K02979.1                                     Ribosome
8  K02117.1                    Oxidative phosphorylation
9  K13018.1  Amino sugar and nucleotide sugar metabolism
Length of dataframe =  5533


In [6]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(563846, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=T

In [7]:
# import os
# import re
# from tqdm.auto import tqdm
# import random

# def find_ko_in_context(Row, directory_path = "/ibex/ai/home/toibazd/annotation_extended", max_files = 10000, context_window=25):
#     embeddings = []
#     label = ko_label_dict[Row["KOs"]]
#     count = 0
#     random.seed(42)
#     for file_name in tqdm(random.sample(os.listdir(directory_path), k=max_files)):
# #         if count>=max_files:
# #             break
#         file_path = os.path.join(directory_path, file_name)
#         with open(file_path, 'r', encoding='latin-1') as file:
#             content = file.read()
#             words = content.split()
#             gene_index = 12
#             while gene_index < len(words)-12:
#                 if words[gene_index] == Row["KOs"]:
#                     start_index = gene_index-12
#                     end_index = gene_index+13
#                     inputs = tokenizer.encode(' '.join(words[start_index:end_index]))
#                     input_ids = torch.tensor([inputs.ids]).to(device)
#                     attention_mask = torch.tensor([inputs.attention_mask]).to(device)

#                     # Generate the embeddings
#                     with torch.inference_mode():
#                         outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states = True)
#                     hidden_states = outputs.hidden_states[-1][0][14]
#                     embeddings.append(hidden_states.cpu())
#                     gene_index += 1 
#                 else:
#                     gene_index += 1
# #         count+=1
#     if embeddings:
#         return (torch.mean(torch.stack(embeddings), dim=0), label)
#     else:
#         print("random returned for label ", label)
#         return (torch.randn(512), label)


# result = df.apply(lambda row: find_ko_in_context(row), axis=1)

# vectors = [item[0] for item in result]

# df['embeddings'] = vectors

In [8]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import random
import torch
from timeit import default_timer as timer


row_number = 0
# Assuming your imports for DataFrame, tokenizer, device, and ko_label_dict are present

def process_file(file_path, keyword):
    try:
        with open(file_path, 'r', encoding='latin-1') as file:
            content = file.read()
            words = content.split()
            embeddings = []
            gene_index = 12
            while gene_index < len(words)-12:
                if words[gene_index] == keyword:
                    start_index = gene_index-12
                    end_index = gene_index+13
                    inputs = tokenizer.encode(' '.join(words[start_index:end_index]))
                    input_ids = torch.tensor([inputs.ids]).to(device)
                    attention_mask = torch.tensor([inputs.attention_mask]).to(device)

                    # Generate the embeddings
                    with torch.inference_mode():
                        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
                    hidden_states = outputs.hidden_states[-1][0][14]
                    embeddings.append(hidden_states.cpu())
                    gene_index += 1
                else:
                    gene_index += 1

            if embeddings:
                return torch.mean(torch.stack(embeddings), dim=0)
            else:
                return torch.randn(512)

    except Exception as e:
        print(f"An error occurred: {e}")
        return torch.randn(512)

def find_ko_in_context_parallel(Row, directory_path="/ibex/ai/home/toibazd/annotation_extended", max_files=70000):
    label = ko_label_dict[Row["KOs"]]
    global row_number
    row_number+=1
    print(label)
    start = timer()
    random.seed(42)
    file_names = random.sample(os.listdir(directory_path), k=max_files)

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_file, os.path.join(directory_path, file_name), Row["KOs"]) for file_name in file_names]

    embeddings = [future.result() for future in as_completed(futures)]
    end = timer()
    print("Taken taken for this row: ", end-start," || Row number: ", row_number)
    if embeddings:
        return (torch.mean(torch.stack(embeddings), dim=0), label)
    else:
        print("random returned for label ", label)
        return (torch.randn(512), label)
    
result_parallel = df.apply(find_ko_in_context_parallel, axis=1)
vectors_parallel = [item[0] for item in result_parallel]

df['embeddings'] = vectors_parallel


Ribosome
Taken taken for this row:  92.05356047581881  || Row number:  1
Ribosome
Taken taken for this row:  89.66007538046688  || Row number:  2
Prokaryotic defense system


KeyboardInterrupt: 

In [ ]:
print(df.head(5))

In [ ]:
df.to_csv("KOs_with_labels_for_BERT.csv", index = False)